In [1]:
import numpy as np 
import scipy as sp  
import random     
import bokeh 
import csv
import pandas as pd 
import struct


In [2]:
#df = pd.read_csv('long.csv')
#sf = pd.read_csv('short.csv')
#d1 = pd.read_csv('test1.csv')
#d2 = pd.read_csv('test2.csv')
#d3 = pd.read_csv('test3.csv') #This is the one I have notes for 
#d4 = pd.read_csv('test4.csv') #Driving Home

#s1 = pd.read_csv('round1.csv')
#s2 = pd.read_csv('round2.csv')
#df = pd.read_csv('round3.csv')
df = pd.read_csv('20170427-stationary-2-leith-croydon.csv')
df.Timestamp = df.Timestamp - df.iloc[0].Timestamp


In [3]:
print max(df.Flags)
# Flags: 0x0040 stationary model --> ego vehicle is stationary?
#        0x0080 detected/validated as mobile
#        0x0100 Validated "valid enough to send out to the interface"
# Always 320 or 448. i.e. only the mobile flag changes.

# Centre of grav = 0
# FL 1
# FR 2
# RR 3
# Rl 4
# FC 5
# RC 6
# LC 8
# ObjC 9
# Ukn FF

#df.ObjExtMeasurement #WTF IS THIS?
#df.ObjPriority -> Priority of what? Imminent threat? Exists between 57344 and 65535.
# df.getRefPointPosCorrCoeffs #Is this is plural, why is it a single float?

#objextmeasurement is a float between 0 and 1
# I don't know what it is.
#Abs / Rel Velocity X/Y
# Is always in the car co-ord frame, only adjusted for Ego Vehicle speed. It is not oriented on the car itself.


448


In [4]:
#motionModelValidated is always true

df.trackedByStationaryModel = (df.trackedByStationaryModel==1)
df.mobile = (df.mobile==1)
df['ObjPrediction'] = (df.ObjectPredAge > 0)

In [5]:
print min(df.trackedByStationaryModel)
print max(df.trackedByStationaryModel)


False
True


In [6]:
print max(df.AbsVelocity_X)

13.2786


In [7]:


from matplotlib import pyplot as plt
Nlines = 260 
from itertools import permutations     
color_lvl = 8     
rgb = np.array(list(permutations(range(0,256,color_lvl),3))) 
rgb_m = np.array(list(permutations(range(0,256,color_lvl),3)))/255.0

from random import sample
colors = sample(rgb,Nlines) 
colors_m = sample(rgb_m,Nlines)

#struct.pack('BBB',*rgb).encode('hex')

#df['color'] = [tuple(colors[i]) for i in df.ObjectId]
df['color'] = ["#" + struct.pack('BBB',*colors[i]).encode('hex') for i in df.ObjectId]
df['color_m'] = [tuple(colors_m[i]) for i in df.ObjectId]

In [8]:
print len(df)
print len(df.where(df['mobile']).color)
x = [None]*len(df)
print len(x)
print len(df[df['mobile']])
print pd.__version__

#Iterator / generator / yield
# Time meory allocation ([None]*len)
# list comprehension [x[i] for i in y]

fill_color= [df['color'].iloc[i] if df['mobile'].iloc[i] else None for i in range(len(df))]

2885759
2885759
2885759
249753
0.18.1


In [9]:
#fill_color= [df['color'].iloc[i] if df['mobile'].iloc[i] else None for i in range(len(df))]
fill_color= [color if flag else None for color, flag in zip(df['color'],df['mobile'])]


In [10]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [11]:
print min(df.ObjExtMeasurement)
print max(df.ObjExtMeasurement)

5.37434e-05
1.0


output_notebook()
df = s3
p = figure(plot_width = 800,plot_height = 800)
for i in df.ObjectId.unique(): 
  color=tuple(colors[i])
  data = df[df.ObjectId==i] 
  p.circle(data.Timestamp,data.ObjectAge,size=2,color=color) 
show(p,notebook_handle=True)

output_notebook()
df = s3
p = figure(plot_width = 800,plot_height = 800)
color=tuple(colors[1])
data = df[df.ObjectId==144] 
p.circle(data.Timestamp,data.ObjectAge,size=2,color=color) 
show(p,notebook_handle=True)

Now, I don't know why I'm looking at disjoint lines. This is supposed to be all observations of 'objectId' 1. So why are not all of the lines starting at zero age?

In [12]:
df.columns

Index([u'ObjectId', u'Flags', u'trackedByStationaryModel', u'mobile',
       u'motionModelValidated', u'ObjectAge', u'Timestamp', u'ObjectPredAge',
       u'Classification', u'ClassCertainty', u'ClassAge', u'ObjBoxCenter_X',
       u'ObjBoxCenter_Y', u'ObjBoxCenterSigma_X', u'ObjBoxCenterSigma_Y',
       u'ObjBoxSize_X', u'ObjBoxSize_Y', u'ObjCourseAngle',
       u'ObjCourseAngleSigma', u'ObjBoxOrientation', u'ObjBoxOrientationSigma',
       u'RelVelocity_X', u'RelVelocity_Y', u'RelVelocitySigma_X',
       u'RelVelocitySigma_Y', u'AbsVelocity_X', u'AbsVelocity_Y',
       u'AbsVelocitySigma_X', u'AbsVelocitySigma_Y', u'RefPointLocation',
       u'RefPointCoords_X', u'RefPointCoords_Y', u'RefPointCoordsSigma_X',
       u'RefPointCoordsSigma_Y', u'RefPointPosCorrCoeffs', u'ObjPriority',
       u'ObjExtMeasurement', u'EgoLatitude', u'EgoLongitude', u'EgoAltitude',
       u'EgoHeadingRad', u'EgoPosTimestamp', u'GPSFixStatus', u'ObjPrediction',
       u'color', u'color_m'],
      dtype='obje

Classes are:
1 Unknown Small
2 Unknown Big
3 Pedestrian
4 Bike
5 Car
6 Truck
The scala also has a 12 id that is bicycle.


output_notebook()
p = figure(plot_width = 800,plot_height = 800)
color=tuple(colors[1])
data = df[df.ObjectId==72] 
#data= data[data.Classification >3]
#The one that starts at 20 something jumps from class 1 to 2
len(colors)
for i in data.Classification.unique(): 
  color=tuple(colors[i*30])
  pf = data[data.Classification==i] 
  p.circle(pf.Timestamp,pf.ObjectAge,size=2,color=color)

#p.circle(data.Timestamp,data.ObjectAge,size=2,color=color) 
show(p,notebook_handle=True)

from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, HoverTool, Div
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.io import curdoc, push_notebook

output_notebook()

time_max = max(df['Timestamp'])
time_min = min(df['Timestamp'])

time_slider = Slider(start=time_min,end=time_max,value=time_min,step=1,title="Time")
source = ColumnDataSource(data=dict(x=[], y=[], color=[], title=[], year=[], revenue=[], alpha=[]))


hover = HoverTool(tooltips=[
    ("ObjectId", "@ObjectId")#,
    #("Year", "@year"),
    #("$", "@revenue")
])

p = figure(plot_height=600, plot_width=700, title="", toolbar_location=None, tools=[hover])
p.circle(x="x", y="y", source=source, size=7, color="color")


def select_data():
    time_val = time_slider.value
    selected = df[
        (df.Timestamp == time_val) 
        #&
        #(movies.BoxOffice >= (boxoffice.value * 1e6)) &
        #(movies.Year >= min_year.value) &
        #(movies.Year <= max_year.value) &
        #(movies.Oscars >= oscars.value)
    ]
    # This is how you do a 'select' widget
    #if (genre_val != "All"):
    #    selected = selected[selected.Genre.str.contains(genre_val)==True]
    
    return selected

def update():
    df = select_data()
    
    source.data = dict(
        x=df['ObjBoxCenter_X'],
        y=df['ObjBoxCenter_Y'],
        color=df['color_m'],
        ObjectId=df['ObjectId'],
        Timestamp=df['Timestamp']
        #title=df["Title"],
        #year=df["Year"],
        #revenue=df["revenue"],
        #alpha=df["alpha"],
    )
    push_notebook()

controls = [time_slider]
for control in controls:
    control.on_change('value', lambda attr, old, new: update())

sizing_mode = 'scale_width'  # 'scale_width' also looks nice with this example

inputs = widgetbox(*controls, sizing_mode=sizing_mode)
l = layout([
    #[desc],
    [p],
    [inputs],
], sizing_mode=sizing_mode)

update()  # initial load of the data

#curdoc().add_root(l)
#curdoc().title = "Intersection"
show(l,notebook_handle=True)
push_notebook()


In [13]:
from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, HoverTool, Div
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.io import curdoc, push_notebook
from ipywidgets import interact
import ipywidgets

output_notebook()

time_max = max(df['Timestamp'])
time_min = min(df['Timestamp'])

time_slider = Slider(start=time_min,end=time_max,value=time_min,step=1,title="Time")
source_rect = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[],angle=[],RefPointLocation=[],
                                    color=[], title=[], year=[], revenue=[], alpha=[]))
source_circle = ColumnDataSource(data=dict(x=[], y=[], line_color=[], fill_color=[],RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
source_crossH = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[],angle=[],RefPointLocation=[],
                                    color=[], title=[], year=[], revenue=[], alpha=[]))
source_crossW = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[],angle=[],RefPointLocation=[],
                                    color=[], title=[], year=[], revenue=[], alpha=[]))
source_ref_pt = ColumnDataSource(data=dict(x=[], y=[], line_color=[], fill_color=[], RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
source_seg = ColumnDataSource(data=dict(x0=[], y0=[], x1=[], y1=[], line_color=[], fill_color=[], RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
source_seg_accel = ColumnDataSource(data=dict(x0=[], y0=[], x1=[], y1=[], line_color=[], fill_color=[], RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
#Sliders:
slider_time = ipywidgets.FloatSlider(
    value = (time_max+time_min)/2, min=time_min, max=time_max, step = 0.1, description="Time",slider_color="red")
window_slider = ipywidgets.FloatSlider(
    value = 5, min=1, max=time_max, step = 0.1, description="Time Window",slider_color="blue")
min_conf_slider = ipywidgets.FloatSlider(
    value = 5, min=0, max=100, step = 1, description="Min Classification Score",slider_color="green")


hover = HoverTool(tooltips=[
    ("ObjectId", "@ObjectId"),
    ("RefPointLocation", "@RefPointLocation"),
    #("$", "@revenue")
])

p = figure(plot_height=600, plot_width=700, title="", tools=[hover],x_range=(-40,20),y_range=(-60,40))
#p = figure(plot_height=600, plot_width=700, title="", tools=[hover],x_range=(-100,0),y_range=(-50,50))
#p.image_url(['intersection3.png'],-78.86,40.10,217.62,128.30)
p.rect(x="x", y="y", width="width",height="height", angle="angle",source=source_crossH, fill_color="color",
       line_color="color")
p.circle(x="x", y="y", source=source_circle, size=4, fill_color="fill_color", line_color="line_color")
p.circle(x="x", y="y", source=source_ref_pt, size=3, fill_color="black", line_color="black")
p.rect(x="x", y="y", width="width",height="height", angle="angle",source=source_rect, fill_color=None,
       line_color="color")
p.rect(x="x", y="y", width="width",height="height", angle="angle",source=source_crossW, fill_color="color",
       line_color="color")
p.segment(x0='x0',y0='y0',x1='x1',y1='y1',line_width=2,line_color='orange',source=source_seg)
p.segment(x0='x0',y0='y0',x1='x1',y1='y1',line_width=2,line_color='pink',source=source_seg_accel)


show(p,notebook_handle=True)

def select_data(time_val, time_window, min_conf):
    #time_val = time_slider.value
    #t_min = time_val-(float(time_window)/2.0)
    #t_max = time_val+(float(time_window)/2.0)
    t_min = time_val-time_window
    t_max = time_val

    selected = df[
        (df.Timestamp > t_min)
        &
        (df.Timestamp < t_max)
        &
        (df.Classification >3 )
        &
        (df.ClassCertainty > min_conf)
    ]
    
    return_df = pd.DataFrame()
    
    for obj_id in selected.ObjectId.unique():
        object_df = selected[selected.ObjectId==obj_id].sort_values(['Timestamp'],ascending=False)
        selected_record=object_df.iloc[0].copy()
        if len(object_df) > 1:
            selected_record['AbsAccel_X'] = object_df['AbsVelocity_X'].iloc[0] - object_df['AbsVelocity_X'].iloc[1]
            selected_record['AbsAccel_Y'] = object_df['AbsVelocity_Y'].iloc[0] - object_df['AbsVelocity_Y'].iloc[1]
        else:
            selected_record['AbsAccel_X'] = 0
            selected_record['AbsAccel_Y'] = 0
        return_df = return_df.append(selected_record)
        
    # Only draw box if car was seen in last 0.5 seconds. 
    # I really only want if it was seen at THIS timestep, but this is close enough
    recent_only_df = return_df[return_df['Timestamp']> (time_val-0.2)]
    return recent_only_df, selected #selected

def update(time_value=slider_time,time_window=window_slider,min_conf=min_conf_slider):
    df_rect, df_circle = select_data(time_value,time_window,min_conf)
    #print df_rect
    source_rect.data = dict(
        x=df_rect['ObjBoxCenter_X'],
        y=df_rect['ObjBoxCenter_Y'],
        width=df_rect['ObjBoxSize_X'],
        height=df_rect['ObjBoxSize_Y'],
        angle=df_rect['ObjBoxOrientation'],
        color=df_rect['color'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    source_ref_pt.data = dict(
        x=df_rect['RefPointCoords_X'],
        y=df_rect['RefPointCoords_Y'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    source_seg.data = dict(
        x0=df_rect['ObjBoxCenter_X'],
        y0=df_rect['ObjBoxCenter_Y'],
        x1=df_rect['ObjBoxCenter_X'].add(df_rect['AbsVelocity_X']),
        y1=df_rect['ObjBoxCenter_Y'].add(df_rect['AbsVelocity_Y']),
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    source_seg_accel.data = dict(
        x0=df_rect['ObjBoxCenter_X'],
        y0=df_rect['ObjBoxCenter_Y'],
        x1=df_rect['ObjBoxCenter_X'].add(10*df_rect['AbsAccel_X']),
        y1=df_rect['ObjBoxCenter_Y'].add(10*df_rect['AbsAccel_Y']),
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    source_circle.data = dict(
        x=df_circle['ObjBoxCenter_X'],
        y=df_circle['ObjBoxCenter_Y'],
        fill_color= [color if not flag else "#FFFFFF" for color,flag in zip(df_circle['color'],df_circle['ObjPrediction'])],
        line_color=df_circle['color'],
        ObjectId=df_circle['ObjectId'],
        #RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_circle['Timestamp']
    )
    source_crossW.data = dict(
        x=df_rect['ObjBoxCenter_X'],
        y=df_rect['ObjBoxCenter_Y'],
        width=df_rect['ObjBoxCenterSigma_X'],
        height=[0.1]*len(df_rect),
        angle=df_rect['ObjBoxOrientation'],
        color=df_rect['color'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    #'ObjBoxCenterSigma_Y'
    source_crossH.data = dict(
        x=df_rect['ObjBoxCenter_X'],
        y=df_rect['ObjBoxCenter_Y'],
        width=[0.1]*len(df_rect),
        height=df_rect['ObjBoxCenterSigma_Y'],
        angle=df_rect['ObjBoxOrientation'],
        color=df_rect['color'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    
    push_notebook()


update(time_max,1,5)  # initial load of the data



Loading BokehJS ...

In [14]:
#If this fails run $ jupyter nbextension enable --py --sys-prefix widgetsnbextension

interact(update,time_val=slider_time,time_window=window_slider,min_conf=min_conf_slider)


Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.update>

In [15]:
time_max


1272.0396428108215